In [3]:
import librosa
import soundfile as sf
import os
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import rfft, irfft, fftfreq, fft, rfftfreq, ifft
from scipy import signal
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import seaborn as sb

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture as GMM
from IPython.display import Audio
import python_speech_features as mfcc
from sklearn import preprocessing

In [9]:
def calculate_delta(array):
    rows, cols = array.shape
    deltas = np.zeros((rows, 20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i - j < 0:
                first = 0
            else:
                first = i - j
            if i + j > rows - 1:
                second = rows - 1
            else:
                second = i + j
            index.append((second, first))
            j += 1
        deltas[i] = (array[index[0][0]] - array[index[0][1]] + (2 * (array[index[1][0]] - array[index[1][1]]))) / 10
    return deltas

def extract_features(audio, rate):
    mfcc_feature = mfcc.mfcc(audio, rate, 0.025, 0.01, 20, nfft=1200, appendEnergy=True)
    mfcc_feature = preprocessing.scale(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature, delta))
    return combined



In [4]:
audio , sr = librosa.load("D:/My PC/Projects/DSP/Voice-Recognition-System/vrs-server/apis/operating.wav")

In [15]:
mfcc_feature = mfcc.mfcc(audio, sr, 0.025, 0.01, 20, nfft=1200, appendEnergy=True)
mfcc_feature

array([[-11.00147748, -11.38914763, -10.31283477, ...,   0.02043865,
         -1.63380095,  -0.24656912],
       [ -9.81863555,  -5.54769192,  -0.49589006, ...,  -3.49370826,
          2.04526672,  -1.61276928],
       [ -9.35784138,  -7.21872803,  -2.75698383, ...,  -3.31973907,
          2.72262115,  -3.84791742],
       ...,
       [-12.37579473,  -0.94456399,  10.89720143, ...,  -1.12773728,
         -3.63915005,   1.15669907],
       [-12.06800837,   0.49684137,  14.41798452, ...,  -1.93619499,
         -5.22528539,   0.63424446],
       [-12.49570662,   1.08088807,  15.09407071, ...,  -2.49154645,
         -6.23143037,   0.2544087 ]])

In [16]:
array = mfcc_feature
rows, cols = array.shape
rows, cols

(270, 20)

In [19]:
deltas = np.zeros((rows, 20))
deltas

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
N = 2
for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i - j < 0:
                first = 0
            else:
                first = i - j
            if i + j > rows - 1:
                second = rows - 1
            else:
                second = i + j
            index.append((second, first))
            j += 1
        deltas[i] = (array[index[0][0]] - array[index[0][1]] + (2 * (array[index[1][0]] - array[index[1][1]]))) / 10
deltas

array([[ 0.44701141,  1.41822949,  2.49286466, ..., -1.01945023,
         1.23919119, -0.85688968],
       [ 0.47541117,  3.01647205,  4.74124115, ..., -1.0644896 ,
         1.30428173, -1.45667853],
       [ 0.33565721,  2.47164721,  3.55582344, ..., -0.60812563,
         0.16751348, -1.58502887],
       ...,
       [ 0.11282526,  0.4532195 ,  1.33854667, ..., -1.19858737,
        -0.44187493, -0.44705617],
       [-0.01326336,  0.62830073,  1.45902544, ..., -0.8631943 ,
        -0.64964622, -0.53771257],
       [-0.0667522 ,  0.46349508,  0.90698247, ..., -0.32829698,
        -0.61907056, -0.21844165]])

In [10]:
extract_features(audio, sr)

array([[-0.5092952 , -0.55806187, -1.06936772, ..., -0.24611208,
         0.30272876, -0.25283244],
       [-0.24671817, -0.14795399, -0.15677692, ..., -0.25698532,
         0.31863008, -0.42980514],
       [-0.14442727, -0.26527151, -0.36696995, ..., -0.14681154,
         0.04092278, -0.46767598],
       ...,
       [-0.81437752,  0.1752153 ,  0.9023337 , ..., -0.28935873,
        -0.10794803, -0.13190765],
       [-0.74605255,  0.27641125,  1.22962843, ..., -0.20838932,
        -0.15870561, -0.15865657],
       [-0.84099655,  0.3174151 ,  1.29247792, ..., -0.0792563 ,
        -0.15123612, -0.06445303]])

In [48]:
def train(csvs):
    models = list()
    for csv in csvs:
        df = pd.read_csv(csv)
        X = df.drop(columns='label', axis=1)


        cols = X.columns
        X = df.drop(columns=[cols[0],'label'], axis=1)
        cols = X.columns
        min_max_scaler = preprocessing.MinMaxScaler()
        np_scaled = min_max_scaler.fit_transform(X)
        # new data frame with the new scaled data.
        X = pd.DataFrame(np_scaled, columns=cols)
        gmm = GMM(n_components=8).fit(X)
        models.append(gmm)

    return models


In [49]:
def score(models, audio_path):

    audio, sr = librosa.load(audio_path, duration=2)
    audio = librosa.effects.trim(audio)
    audio = audio[0]

    features = [librosa.feature.chroma_stft(y=audio, sr=sr).mean(), librosa.feature.chroma_stft(y=audio, sr=sr).var(),
                librosa.feature.rms(y=audio).mean(), librosa.feature.rms(y=audio).var(),
                librosa.feature.spectral_centroid(y=audio).mean(),librosa.feature.spectral_centroid(y=audio).var(),
                librosa.feature.spectral_bandwidth(y=audio).mean(),librosa.feature.spectral_bandwidth(y=audio).var(),
                librosa.feature.spectral_rolloff(y=audio).mean(),librosa.feature.spectral_rolloff(y=audio).var()]

    mfcc = librosa.feature.mfcc(y=audio)
    mfcc_list = []
    for i in range(len(mfcc)):
         mfcc_list.append(mfcc[i].mean())


    features = features + mfcc_list
    score_list = np.zeros(len(models))
    for i in range(len(models)):
        out = np.array(models[i].score([features]))
        score_list[i] = out.sum()

        # score_list.append(model.score([features]))
    return score_list

In [54]:
csvs = ['Mariam.csv','Amr.csv','Momen.csv','ibrahim.csv']
models = train(csvs)
myScoreList = score(models, 'D:/My PC/Projects/DSP/Voice-Recognition-System/Model/Website Data/Ibrahim/10.wav')
print(myScoreList)
winner = np.argmax(myScoreList)
print(winner)

[-4.61565422e+18 -4.31904848e+18 -3.77256385e+18 -4.28236374e+18]
2


C:\Users\I1bra\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\Users\I1bra\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\Users\I1bra\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\Users\I1bra\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
